## Parametric FDC Estimation

Consider a sequence of methods for estimating the flow duration curve at an unmonitored location. Starting from the simplest method and increasing in complexity.  The goal is to estimate a distribution, then evaluate the expectation of the method over a large sample by computing the divergence between the "true" distribution $P$ and the corresponding estimate $Q$.  Consider several divergences from the class of f-divergences for their unique properties, namely their associated conjugate dual or surrogate loss functions. 

>**Note**: clarify the relationship between individual f-divergences, their conjugate dual functions, and their surrogate loss functions.

1.  **Naive or maximum uncertainty**: as a baseline, the divergence of a probability distribution from the uniform distribution is related to the entropy.  This may be interpretable as a baseline that describes the maximum reduction in uncertainty, i.e. from maximum uncertainty to observation.  (we may take some kind of result from asymptotic convergence towards the true entropy/divergence from the Jordan results as a way to describe "uncertainty about uncertainty")  *"This problem includes as a special case the problem of estimating the mutual information, corresponding to the KL divergence between a joint distribution and the product of its marginals, as well as the problem of estimating the Shannon entropy of a distribution P, which is related to the KL divergence between P and the uniform distribution."* (see Nguyen et al. 2009 **Estimating divergence functionals and the likelihood  ratio by convex risk minimization**) 
2.  **k-Nearest neighbour**: take the k nearest streamflow stations and approximate the target location based on some kind of average.  Subsets of this method include a) different k, b) weighting schemes on top of different k, or c) other intermediate models on k.
3.  **"Best" neighbour**: how much better could the "best" neighbour be if we knew how to pick it?  Note: we don't really know how to pick it. (see Poole et al. 2021 Regionalization for Ungauged Catchments — Lessons Learned From a Comparative Large-Sample Study).
    * An "upper" bound for this is computing what the "extra bits per sample" cost of using a uniform distribution as a model, i.e. $D_K(P||U)$ because some number of neighbours will be worse than this (one question is now many).
    * A "lower" bound for this is estimating the uncertainty in the distribution estimate itself, because there is a limit to the precision we can use to rank neighbours based on KL divergence.  
5.  **Parametric distribution estimation**: (following previous results) we show that the mean runoff can be predicted well.  We know that the standard deviation is well correlated to the mean, so we have two parameters that we can use to estimate the distribution using any number of parametric forms. **Non-parametric distributions do not represent generators, so KDE can't be used to estimate a distribution for an ungauged catchment**.
6.  **Divergence Prediction**: what if we ignore the specifics how *how* the two distributions differ and instead focus on *how much* they differ.


One model that predicts sufficient statistics, then compute KL divergence.
Next model that predicts the computed KL divergence.
Both models use only catchment characteristics.

## Methodology

Let $P$ represent the "ground truth" distribution of unit area runoff for a monitored catchment $i$.  Let $\hat P_i$ represent an estimate of $P_i$.

For all $i$ in $M$ monitored catchments:

1. Compute an estimate of the entropy of $P_i$ based on $\hat P_i$:
    * The entropy is sensitive to the method 
3. Estimate
4. Use the parameters as a model Q in pairwise comparisons, these eliminate

In [1]:
import os
from time import time
from scipy.stats import entropy
import multiprocessing as mp
import data_processing_functions as dpf
import numpy as np
import pandas as pd
import geopandas as gpd

# visualize the catchment centroid locations
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, LinearAxis, Range1d
from bokeh.palettes import Colorblind, Sunset10

from scipy.stats import lognorm, expon, kappa4, gaussian_kde
from scipy.special import kl_div, digamma
from scipy.optimize import minimize

# from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cdist

from KDEpy import FFTKDE
import jax.numpy as jnp
from jax import vmap
from jax.scipy.stats import gaussian_kde as jkde

output_notebook()

BASE_DIR = os.getcwd()

Loading BokehJS ...

In [3]:
# import attributes 
rev_date = '20250227'
attributes_fpath = os.path.join(os.path.join('data', f'BCUB_watershed_attributes_updated_{rev_date}.geojson'))
attr_df = gpd.read_file(attributes_fpath)
attr_df.columns = [e.lower() for e in attr_df.columns]
filtered_stns = sorted(list(set(attr_df['official_id'].values)))

In [ ]:
# load the predicted parameter results
predict_result_folder = '/home/danbot2/code_5820/24/divergence_measures/docs/notebooks/data/prediction_results/runoff_prediction_results'
# target_cols = ['mean_uar', 'sd_uar', 'logq_mean_mm', 'logq_sd_mm',]
target_columns = ['mean_uar', 'sd_uar', 'mean_logx', 'sd_logx']

predicted_param_dict = {}
for t in target_columns:
    print(t)
    file = f'best_out_of_sample_{t}_predictions.csv'
    rdf = pd.read_csv(os.path.join(predict_result_folder, file), index_col='official_id')
    rdf = rdf[[c for c in rdf.columns if not c.startswith('Unnamed:')]].sort_values('official_id')
    predicted_param_dict[t] = rdf.to_dict(orient='index')

## General data processing functions

In [17]:
def generate_bootstrap_samples(data, N):
    """
    Generate a matrix of N bootstrap samples, each of length L.

    Parameters:
        data (array-like): 1D array of sample data.
        N (int): Number of bootstrap samples to generate.
        L (int): Length of each bootstrap sample.

    Returns:
        np.ndarray: Matrix of shape (N, L) containing bootstrap samples.
    """
    data = np.array(data)
    indices = np.random.randint(0, len(data), size=(len(data), N))
    bootstrap_samples = data[indices]
    return bootstrap_samples

## Estimate entropy

The entropy of a distribution $P$ is related to the KL divergence between $P$ and the uniform distribution $\mathbb{U}$ by the following:

The KL divergence between two distributions $P$ and $Q$ is:

$$D_\text{KL}(P||Q) = \int p(x) \log \frac{p(x)}{q(x)} dx$$

Where $p(x)$ and $q(x)$ are the probability density functions of $P$ and $Q$ respectively.  If $Q$ is a uniform distribution over $[a, b]$, the density is the constant $q(x) = (b-a)^{-1}$, $x \in [a, b]$.  Substituting into the KL divergence expression:

$$D_\text{KL}(P||Q) = \int p(x) \log \frac{p(x)}{1/(b-a)}dx = \int p(x) \log p(x)dx - \int p(x)\log \frac{1}{b-a}dx$$

Since $log\frac{1}{b-a}$ is constant over the domain,

$$\int p(x)\log \frac{1}{b-a} dx = \log(b-a)$$

And the first term is the negative entropy of $p(x)$, i.e. $-H(P) = \int p(x) \log (x) dx$, then:

$$D_\text{KL}(P||Q) = -H(P) + \log(b-a)$$

So the maximum uncertainty $\log(b-a)$ is reduced by $\log(b-a) - H(P) = D_\text{KL}(P||Q)$ bits per sample by observing the system. 

## Define a measurement error model



In [21]:
# Define the ranges and associated errors
error_points = jnp.array([0.0001, 0.001, 0.01, 0.1, 1.0, 10, 1e2, 1e3, 1e4, 1e5])  # Magnitude points
error_values = jnp.array([10., 5.0, 1., 0.5, 0.25, 0.1, 0.1, 0.15, 0.20, 0.25])    # Associated errors (as proportions)

efig = figure(width=400, height=300, x_axis_type='log')
efig.line(error_points, error_values, line_color='red', line_width=2, legend_label='Measurement Error Model')
efig.xaxis.axis_label = r'$$\text{Flow } m^3/s$$'
efig.yaxis.axis_label = r'$$\text{Error } [/100]$$'
efig = dpf.format_fig_fonts(efig, font_size=12)
show(efig)

In [25]:
def vectorized_kde_integration(log_data, eval_grid):
    """
    Perform integration for a single bootstrap sample using its KDE fit.
    
    Parameters:
    - log_data: Log-transformed data points for one bootstrap sample (1D array).
    - eval_grid: Shared evaluation grid (1D array).

    Returns:
    - Integrated densities over the evaluation grid (1D array).
    """
    fit_object = jkde(log_data, bw_method='silverman')
    return vmap(lambda le: fit_object.integrate_box_1d(-jnp.inf, le))(eval_grid)


def compute_baseline_kde_fits(eval_grid, bootstrap_samples):
    """
    Compute KDE for each bootstrap sample column in a vectorized manner.
    
    Parameters:
    - eval_grid: Shared evaluation grid (array of shape (n_eval,)).
    - bootstrap_samples: Matrix of bootstrap samples (shape (n_data, n_bootstrap)).
    
    Returns:
    - Density estimates for each bootstrap sample (array of shape (n_eval, n_bootstrap)).
    """
    # Log-transform the bootstrap samples
    log_samples = jnp.log10(bootstrap_samples)

    # Define a function to integrate a single column
    def integrate_column(log_data):
        return vectorized_kde_integration(log_data, jnp.array(eval_grid))
    
    # Apply the function to each column of the bootstrap samples
    cdf = vmap(integrate_column, in_axes=1, out_axes=1)(log_samples)
    pmf = jnp.diff(cdf, axis=0)
    pmf /= np.sum(pmf, axis=0)
    return pmf

In [26]:
# Interpolated error function
def interpolate_error_model(x):
    """
    Interpolate measurement error as a function of x using the provided error ranges.
    """
    return jnp.interp(x, error_points, error_values)

def compute_fft_kde_fit_bootstrap(data, grid):
    # input_data = jnp.log10(data)
    fft_weights = jnp.asarray(vmap(interpolate_error_model)(data)).copy()

    fft_weights /= fft_weights.sum(axis=0)
    
    log_data = np.log10(data) 
    cdfs = []
    nn = 0
    for d, w in zip(log_data.T, fft_weights.T):
        nn += 1
        fft_fit = FFTKDE().fit(np.array(d), weights=np.array(w))
        x, y = fft_fit.evaluate()
        fft_density = fft_fit.evaluate(grid)
        cdfs.append(fft_density)
        
    fft_kde_cdf = jnp.cumsum(jnp.array(cdfs).T, axis=0) * (grid[1] - grid[0])
    # Normalize CDF to ensure it sums to 1
    fft_kde_cdf /= fft_kde_cdf[-1]
    pmf = np.diff(fft_kde_cdf, axis=0)
    pmf /= np.sum(pmf, axis=0)
    return pmf

In [12]:
n_simulations = 50
n = 1
max_diff = 0
results = []
for stn in filtered_stns:
    test_df = dpf.get_timeseries_data(stn)
    test_df.dropna(subset=[stn], inplace=True)
    data = test_df[stn].dropna().values
    data = add_jitter(data)
    samples = generate_bootstrap_samples(data, n_simulations)
    log_samples = np.log10(samples)

    # create a common grid to evaluate pdfs numerically
    eval_grid = create_pdf_eval_grid(data)

    # here we generate bootstrap samples of the baseline 
    # distribution estimate
    kde_fits = compute_baseline_kde_fits(eval_grid, samples) 
    # print(kde_fits.shape)

    # compute distributions incorporating adaptive kernel
    # bandwidth based on an assumed measurement error model
    adaptive_kde_fits = compute_fft_kde_fit_bootstrap(samples, eval_grid)
    # print(adaptive_kde_fits.shape)
    # print(asdf)
    # compute the KL divergence between simple and weighted KDE fits
    # for the bootstrap samples
    kde_KLD = compute_kl_divergence(kde_fits, adaptive_kde_fits)
    kde_CI = np.percentile(kde_KLD, (2.5, 50, 97.5))
    results.append(kde_CI)
    ci = kde_CI[2] - kde_CI[0]

    # find the nearest k stations and use their distribution(s) as a model
    # for k in [1, 2, 3, 4, 5]:
        
    
    # uncertainty_reduction_kde = np.round(max_uncertainty - np.percentile(kde_H, (97.5, 50, 2.5)), 2)
    # ci = uncertainty_reduction_kde[2] - uncertainty_reduction_kde[0]
    # results.append([ci, len(test_df)])
    if ci > max_diff:
        max_diff = ci
        print(f'New largest CI = {ci:.2f} (N={len(test_df)})')
    if n % 25 == 0:
        print(f'    {n}/{len(filtered_stns)} completed')
        # print(asdf)
    n += 1

New largest CI = 0.01 (N=1752)


2025-01-16 12:51:44.003774: W external/xla/xla/tsl/framework/bfc_allocator.cc:497] Allocator (GPU_0_bfc) ran out of memory trying to allocate 459.39MiB (rounded to 481702912)requested by op 
2025-01-16 12:51:44.004051: W external/xla/xla/tsl/framework/bfc_allocator.cc:508] ****************************************************************************************__*********_
E0116 12:51:44.004086 2110204 pjrt_stream_executor_client.cc:3085] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 481702800 bytes.


ValueError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 481702800 bytes.

In [ ]:
ci_df = pd.DataFrame(results, columns=['ci', 'n'])
fig = figure(title=f'KDE estimated H(P) CI vs. sample size, {n_simulations} bootstrap samples')#, x_axis_type='log')
years_equiv = ci_df['n'] / 365.24
fig.scatter(years_equiv, ci_df['ci'], size=2, legend_label='95% CI')
fig.xaxis.axis_label = r'$$\text{Sample size [x365]}$$'
fig.yaxis.axis_label = r'$$\text{[bits/sample]}$$'
fig = dpf.format_fig_fonts(fig)
show(fig)

The idea behind the above plot is to show how the uncertainty, as expressed by the 95% confidence interval of a bootstrap sample, in the entropy estimate of a large sample of arbitrary distributions, varies as a function of the sample size.  The x-axis represents the "effective" sample size in more intuitive terms of years.  

The motivation is to understand how the length of record is reflected in downstream analysis when comparing pairs of arbitrary distributions.

What is needed is to figure out how to convey this in terms that relate to the ultimate discriminant, or the ability to distinguish between potential models for their reduction in uncertainty.  Stated otherwise, to describe some kind of threshold where we can say that one proxy model is better than another, that is provides a more accurate estimate of the 'ground truth' distribution.  Such a model would enable the large scale comparison of potential future observation network expansion locations on the basis of how much they reduce the **aggregate uncertainty** of the unmonitored space.

## 

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

def neg_log_likelihood(params):
    h, k, loc, scale = params
    if scale <= 0:
        return np.inf  # Enforce positive scale
    try:
        logpdf = kappa4.logpdf(data, h, k, loc=loc, scale=scale)
        if np.any(np.isnan(logpdf) | np.isinf(logpdf)):
            return np.inf
        return -np.sum(logpdf)
    except Exception:
        return np.inf

def fit_kappa4_mle(data):
    if len(data) < 2 or np.std(data) <= 0:
        # Handle edge cases where MLE is not feasible
        return {'h': np.nan, 'k': np.nan, 'loc': np.nan, 'scale': np.nan}
    
    # Define the initial guesses for the parameters
    initial_params = [0.0, 0.0, np.mean(data), np.std(data)]
    
    # Bounds for the parameters: h and k between -2 and 2, scale > 1e-5
    bounds = [(-2, 2), (-2, 2), (None, None), (1e-5, None)]
    
    # Minimize the negative log-likelihood
    result = minimize(neg_log_likelihood, initial_params, method='L-BFGS-B', bounds=bounds)
    
    if result.success:
        h, k, loc, scale = result.x
    else:
        h, k, loc, scale = np.nan, np.nan, np.nan, np.nan
        # Log or handle fitting failure if needed
    
    return {'h': h, 'k': k, 'loc': loc, 'scale': scale}


def compute_pdf_cdf_kde_scipy(data, grid_points=1000, bandwidth=None):
    """
    Computes the probability density function (pdf) and cumulative distribution function (cdf)
    from an array of values using Kernel Density Estimation (KDE) with scipy.stats.

    Parameters:
    data (array-like): Input data array.
    grid_points (int): Number of points in the grid where the pdf and cdf are evaluated.
    bandwidth (float or str, optional): The bandwidth of the kernel. If None, Scott's Rule is used.
                                         Can also be a string for methods like 'scott' or 'silverman'.

    Returns:
    x_grid (numpy.ndarray): Grid points where the pdf and cdf are evaluated.
    pdf_values (numpy.ndarray): Estimated pdf values corresponding to x_grid.
    cdf_values (numpy.ndarray): Estimated cdf values corresponding to x_grid.
    """
    # Convert input data to a numpy array
    data = np.asarray(data)
    
    # Create a Gaussian KDE object
    kde = gaussian_kde(data, bw_method=bandwidth)
    
    # Create a grid over which to evaluate the KDE
    x_min = data.min() - 1.0 * data.std()
    x_max = data.max() + 1.0 * data.std()
    x_grid = np.linspace(x_min, x_max, grid_points)
    
    # Evaluate the pdf over the grid
    pdf_values = kde.evaluate(x_grid)
    
    # Compute the cdf by integrating the pdf
    cdf_values = np.array([kde.integrate_box_1d(-np.inf, xi) for xi in x_grid])
    
    return x_grid, pdf_values, cdf_values

In [ ]:
def remap_low_to_high_resolution(data, b1, b2, which_binning='log'):
    """
    Vectorized distribution of low-resolution frequencies over high-resolution bins.
    Expands each low-res bin's frequency uniformly across the corresponding high-res bins.
    """
    b1_bin_dict = compute_discrete_distributions(data, b1)  # Low-res (Q)
    b2_bin_dict = compute_discrete_distributions(data, b2)  # High-res (P)
    b1_bins = b1_bin_dict[which_binning]['edges']
    b1_freqs = b1_bin_dict[which_binning]['freqs']
    b2_bins = b2_bin_dict[which_binning]['edges']
    b2_freqs = b2_bin_dict[which_binning]['freqs']
    # Normalize up-scaled low-res frequencies and convert to probabilities
    b1_probs = b1_freqs / np.sum(b1_freqs)
    # Determine the low-res bin for each high-res bin
    bin_indices = np.digitize(b2_bins[:-1], b1_bins) - 1  # Map high-res bins to low-res bins
    
    # Compute counts of high-res bins falling into each low-res bin
    counts_per_low_bin = np.bincount(bin_indices, minlength=len(b1_probs))

    # Broadcast low-res frequencies to high-res bins, dividing by the count to distribute uniformly
    high_res_probs = b1_probs[bin_indices] / counts_per_low_bin[bin_indices]
    # Normalize to ensure the result sums to 1
    high_res_probs /= np.sum(high_res_probs)
    
    return high_res_probs, b1_bin_dict, b2_bin_dict

In [ ]:
def compute_discrete_distributions(data, b):
    # 1. Determine the log-spaced bin edges
    bin_dict = {}
    total_count = len(data)
    # 1. Compute the histogram using equal-width bins
    counts, bin_edges = np.histogram(data, bins=2**b, density=False)
    densities, _ = np.histogram(data, bins=2**b, density=True)
    freqs = counts / len(data)
    bin_widths = bin_edges[1:] - bin_edges[:-1] 
    bin_dict['equal'] = {'edges': bin_edges, 'freqs': freqs, 'densities': densities, 'widths': bin_widths}

    # 2. Compute the histogram using log-spaced bins
    minx, maxx = np.min(data), np.max(data)
    log_edges = np.logspace(np.log10(minx), np.log10(maxx), 2**b + 1)
    bin_widths = log_edges[1:] - log_edges[:-1]
    log_densities, _ = np.histogram(data, bins=log_edges, density=True)
    log_counts, _ = np.histogram(data, bins=log_edges, density=False)
    log_freqs = log_counts / sum(log_counts)
    assert abs(sum(log_freqs) - 1) < 0.001, sum(log_freqs)
    bin_dict['log'] = {'edges': log_edges, 'freqs': log_freqs, 'densities': densities, 'widths': bin_widths}

    # 3. Compute the histogram using uniform (probability) bins
    quantiles = np.linspace(0, 1, 2**b + 1)
    uniform_edges = np.quantile(data, quantiles)
    bin_widths = uniform_edges[1:] - uniform_edges[:-1]
    uniform_freqs, _ = np.histogram(data, bins=uniform_edges, density=True)
    uniform_counts, _ = np.histogram(data, bins=uniform_edges, density=False)
    uniform_freqs = uniform_counts / total_count
    # assert abs(sum(uniform_freqs) - 1) < 0.001, sum(uniform_freqs)
    bin_dict['uniform'] = {'edges': uniform_edges, 'freqs': uniform_freqs, 'densities': densities, 'widths': bin_widths}
    return bin_dict

In [ ]:
def create_MLE_fit_plot(b, df, stn):
    test_fig = figure(title=None,
                     width=600, height=500, x_axis_type='log')  

    # plot empirical (discrete) distributions using linear and log binning
    b=8
    label = f'{b}_bit_log'
    bin_edges, freqs, log_bin_edges, log_freqs = compute_discrete_distributions(df, b, label, stn)
    test_fig.quad(left=bin_edges[:-1], right=bin_edges[1:], top=freqs, bottom=[0 for _ in freqs], 
                  legend_label=f'{b }bits linear bins', color=Sunset10[0], fill_alpha=0.4, line_color=None)
    test_fig.quad(left=log_bin_edges[:-1], right=log_bin_edges[1:], top=log_freqs, 
                  bottom=[0 for _ in freqs], legend_label=f'{b} bits log bins', color=Sunset10[8], 
                  line_color=None, fill_alpha=0.6)

    # fit and plot a lognormal distribution
    ln_shape, ln_loc, ln_scale = lognorm.fit(df[stn], floc=0)  # Fixing location to 0
    x = np.logspace(-2, 3, 1000)
    ln_mle_pdf = lognorm.pdf(x, ln_shape, loc=0, scale=ln_scale)

    # fit and plot an exponential distribution
    ex_loc, ex_scale = expon.fit(df[stn], floc=0)
    ex_mle_pdf = expon.pdf(x, loc=0, scale=ex_scale)

    test_fig.line(x, ln_mle_pdf, color='black', legend_label='LN MLE pdf', line_width=2)
    test_fig.line(x, ex_mle_pdf, color='grey', legend_label='EXP MLE pdf', line_width=2)
    test_fig.legend.background_fill_alpha = 0.6
    test_fig.legend.location = 'top_right'
    test_fig.legend.click_policy='hide'
    test_fig.xaxis.axis_label = r'$$\text{Mean Daily Flow } [m^3/s]$$'
    test_fig.yaxis.axis_label = r'$$P(X)$$'
    test_fig = dpf.format_fig_fonts(test_fig)
    return test_fig


In [ ]:
def create_logspace_bins(data, num_bins):
    """Create linearly spaced bins in log space over the range of `data`."""
    min_val, max_val = np.min(data), np.max(data)
    log_min, log_max = np.log10(min_val), np.log10(max_val)
    bin_edges = np.logspace(log_min, log_max, num_bins + 1)
    return bin_edges


def calculate_probabilities(data, bin_edges):
    """Calculate the probabilities of data falling into each bin given `bin_edges`."""
    counts, _ = np.histogram(data, bins=bin_edges)
    probabilities = counts / counts.sum()  # Normalize to get probabilities
    return probabilities


def aggregate_probabilities(high_res_probs, high_res_bins, low_res_bins):
    """
    Aggregate high-resolution probabilities to match low-resolution bins.
    This will sum the high_res_probs that fall within each low-res bin.
    """
    low_res_probs = []
    for i in range(len(low_res_bins) - 1):
        # Find indices of high-resolution bins that fall within the current low-res bin
        indices = np.where((high_res_bins[:-1] >= low_res_bins[i]) & (high_res_bins[:-1] < low_res_bins[i+1]))[0]
        # Sum probabilities within the range
        low_res_probs.append(np.sum(high_res_probs[indices]))
    return np.array(low_res_probs)


def compute_kl_divergence(p, q):
    """Compute KL divergence between two probability distributions `p` and `q`."""
    # Avoid division by zero and log of zero by adding a small epsilon
    # epsilon = 1e-10
    # p = np.maximum(p, epsilon)
    # q = np.maximum(q, epsilon)
    kl_divergence = np.sum(p * np.log(p / q))
    return kl_divergence

    
def fit_continuous_distributions(data, bin_edges):
    """simulate the target using the parametric MLE parameters from the proxy"""
    # fit and plot a lognormal distribution
    ln_shape, ln_loc, ln_scale = lognorm.fit(data, floc=0)  # Fixing location to 0
    ex_loc, ex_scale = expon.fit(data, floc=0)
    kp = fit_kappa4_mle(data)
    # kp = constrained_optimization(data)
    kde = gaussian_kde(np.log10(data), bw_method='scott')

    edges = list(bin_edges)# + [np.inf]
    ln_cdf_vals = lognorm.cdf(edges, ln_shape, 
                              loc=ln_loc, scale=ln_scale)
    expon_cdf_vals = expon.cdf(edges, loc=ex_loc, 
                               scale=ex_scale)
    kappa4_cdf_vals = kappa4.cdf(edges, kp['h'], kp['k'],
                                 loc=kp['loc'], scale=kp['scale'])
    
    # Compute the KDE-based CDF at the evaluation points
    kde_cdf_vals = np.array([kde.integrate_box_1d(-np.inf, np.log10(xi)) for xi in edges])
    
    p_sim = pd.DataFrame()
    p_sim[f'{stn}_LN'] = np.diff(ln_cdf_vals)
    p_sim[f'{stn}_EXP'] = np.diff(expon_cdf_vals)
    p_sim[f'{stn}_KP4'] = np.diff(kappa4_cdf_vals)
    p_sim[f'{stn}_KDE'] = np.diff(kde_cdf_vals)
    # normalize the distributions 
    p_sim /= p_sim.sum()
    # make sure all distributions sum to 1
    assert np.isclose(p_sim.sum(), 1, atol=0.0001).all(), p_sim.sum()
    bin_midpoints = (np.array(bin_edges[:-1]) + np.array(bin_edges[1:])) / 2
    # replace the last bin midpoint with half the previous bin's width
    # because our right bin edge is np.inf
    # right_bin_midpoint = edges[-2] + (edges[-2] - edges[-3]) / 2.0
    # bin_midpoints[-1] = right_bin_midpoint
    p_sim['bin_midpoints'] = bin_midpoints
    return p_sim

In [ ]:
def plot_quantization_comparison(b1, b2, df, stn):
    test_fig = figure(title=None,
                     width=600, height=500, x_axis_type='log')  

    # plot empirical (discrete) distributions using linear and log binning
    colors = ['gold', 'green']
    n = 0
    for b in [b1, b2]:
        clr = colors[n]
        label = f'{b}_bit_log'
        bin_edges, freqs, log_bin_edges, log_freqs = compute_discrete_distributions(df, b, label, stn)
        test_fig.quad(left=log_bin_edges[:-1], right=log_bin_edges[1:], top=log_freqs, 
                      bottom=[0 for _ in freqs], legend_label=f'{b} bits log bins', color=clr, 
                      line_color=None, fill_alpha=0.6)
        n += 1
    # fit and plot a lognormal distribution
    ln_shape, ln_loc, ln_scale = lognorm.fit(df[stn], floc=0)  # Fixing location to 0
    x = np.logspace(-2, 3, 1000)
    ln_mle_pdf = lognorm.pdf(x, ln_shape, loc=0, scale=ln_scale)

    kld_test = kl_divergence_between_quantizations(df, b1, b2, stn)
    print(f'KLd between {b1} and {b2} bit quantizations = {kld_test:.2f}')

    # fit and plot an exponential distribution
    ex_loc, ex_scale = expon.fit(df[stn], floc=0)
    ex_mle_pdf = expon.pdf(x, loc=0, scale=ex_scale)
    test_fig.line(x, ln_mle_pdf, color='black', legend_label='LN MLE pdf', line_width=2)
    test_fig.line(x, ex_mle_pdf, color='grey', legend_label='EXP MLE pdf', line_width=2)
    test_fig.legend.background_fill_alpha = 0.6
    test_fig.legend.location = 'top_right'
    test_fig.legend.click_policy='hide'
    test_fig.xaxis.axis_label = r'$$\text{Mean Daily Flow } [m^3/s]$$'
    test_fig.yaxis.axis_label = r'$$P(X)$$'
    test_fig = dpf.format_fig_fonts(test_fig)
    return test_fig

In [ ]:
qn_df = pd.DataFrame.from_dict(quantization_noise_results, orient='index', columns=test_bs)
bounds = pd.DataFrame()
qn_fig = figure(width=700, height=400)
for p in [2.5, 25, 50, 75, 98.5]:
    bounds[p] = [np.percentile(qn_df[c], p) for c in test_bs]
bounds.index = test_bs
qn_fig.varea(x=test_bs, y1=bounds[2.5], y2=bounds[98.5], alpha=0.4, color='grey', legend_label='95% CI')
qn_fig.varea(x=test_bs, y1=bounds[25], y2=bounds[75], alpha=0.4, color='black', legend_label='IQR')
qn_fig.line(x=test_bs, y=bounds[50], color='crimson', legend_label='Median', line_dash='dashed', line_width=3)
qn_fig.yaxis.axis_label = r"$$\text{Noise [bits/sample]}$$"
qn_fig.xaxis.axis_label = r"$$\text{Dictionary Size (b)} [2^b = \text{N symbols}]$$"
qn_fig = dpf.format_fig_fonts(qn_fig)
show(qn_fig)

In [ ]:
stn = filtered_stns[0]
test_df = dpf.get_timeseries_data(stn)
foo = plot_quantization_comparison(4, 5, test_df, stn)

show(foo)

In [ ]:
from bokeh.io import export_png

test_stn = filtered_stns
for stn in filtered_stns:
    output_folder = 'MLE_plots'
    plot_fpath = os.path.join(output_folder, f"{stn}_LN_and_expon_fits.png")
    if os.path.exists(plot_fpath):
        continue
    test_df = dpf.get_timeseries_data(stn)
    test_df.dropna(subset=[stn], inplace=True)
    minx, maxx = test_df[stn].min(), test_df[stn].max()
    # print(f'X range is {minx:.1f} to {maxx:.1f} cms')
    # print('')
    
    test_fig = create_MLE_fit_plot(8, test_df, stn)
    
    export_png(test_fig, filename=plot_fpath)

In [ ]:
n = 0
for stn in filtered_stns:
    df = dpf.get_timeseries_data(stn)
    ln_shape, ln_loc, ln_scale = lognorm.fit(df[stn], floc=0)
    expon_loc, expon_scale = expon.fit(df[stn], floc=0)
    # kh, kk, k_loc, k_scale = fit_kappa4_mle(df[stn])
    # kh, kk, k_loc, k_scale = constrained_optimization(df[stn])
    attr_df.loc[attr_df['official_id'] == stn, ['ln_shape', 'ln_loc', 'ln_scale']] = (ln_shape, ln_loc, ln_scale)
    attr_df.loc[attr_df['official_id'] == stn, ['expon_loc', 'expon_scale']] = (expon_loc, expon_scale)
    # attr_df.loc[attr_df['official_id'] == stn, ['kappa_h', 'kappa_k', 'kappa_loc', 'kappa_scale']] = (kh, kk, k_loc, k_scale)
    for b in test_bs:
        attr_df.loc[attr_df['official_id'] == stn, f'{b}_quantization_noise'] = kl_divergence_between_quantizations(df, b, max(test_bs), stn) 
    n += 1
    if n % 150 == 0:
        print(f'    ...{n}/{len(filtered_stns)} completed.')

In [ ]:
# convert the MLE parameters to dicts for easier access
ln_dict = (
    attr_df
    .set_index('official_id')[['ln_shape', 'ln_loc', 'ln_scale']]
    .to_dict(orient='index')
)
expon_dict = (
    attr_df
    .set_index('official_id')[['expon_loc', 'expon_scale']]
    .to_dict(orient='index')
)

# kappa_dict = (
#     attr_df
#     .set_index('official_id')[['kappa_a', 'kappa_b', 'kappa_c', 'kappa_d']]
#     .to_dict(orient='index')
# )

## Compute the "noise" added to a discriminant value due to assuming an error distribution as a prior

Rating curve uncertainty is a hard problem in hydrology.  Instead of treating daily flow observations as discrete measurements with the fixed (often overzealous) precision that it is published by governing agencies, we can assume some kind of basic error model and test how much the error model distorts the information in the distribution.  In other words, how much noise/uncertainty is added for any model error.  

Below we'll test a range of uniform error distributions as models for the observations.  We'll take an example streamflow record, and we'll quantize it to a range of dictionary sizes in two ways.  One way is to bin the observations as they are, we'll refer to this as the "deterministic" treatment.  The second way is to apply a series of error distribution models, calling it the "stochastic treatment", and bin the observations by counting the fraction of the observation distribution interval that lies in each bin.  In other words, we'll count partial observations in proportion to where they fall over the binning intervals as opposed to counting a whole observation based on the interval alone.

The quantization will take in a bitrate $b$, and it will divide and log-transform the measured interval $(\log(x_\text{min}),\log(x_\text{max}))$ into $2^b$ log-spaced bins.  

In [ ]:
def compute_log_uniform_bins(df, stn, bitrate):
    n_bins = 2**bitrate
    min_log_val = np.log10(df[stn].min())
    max_log_val = np.log10(df[stn].max())

    # set the bin edges to be evenly spaced between the
    # observed range of the proxy/donor series
    # np.digitize will assign 0 for out-of-range values at left
    # and n_bins + 1 for out-of-range values at right
    log_bin_edges = np.linspace(
        min_log_val,
        max_log_val,
        n_bins + 1,
    ).flatten()

    # convert back to linear space
    bin_edges = [10**e for e in log_bin_edges]

    # there should be n_bins edges which define n_bins - 1 bins
    # this is to reserve 2 bin for out-of-range values to the right
    assert len(bin_edges) == n_bins + 1
    return bin_edges

In [ ]:
def apply_error_to_observations(df, stn, bitrate=None, error=0.1):
    min_q, max_q = df[stn].min() - 1e-9, df[stn].max() + 1e-9
    assert min_q > 0
    # use equal width bins in log10 space
    bin_edges = compute_log_uniform_bins(df, stn, bitrate)
    # df[f'{bitrate}_bits_quantized'] = np.digitize(df[stn], bin_edges)
    fractional_obs_counts = dpf.error_adjusted_fractional_bin_counts(
        df[stn], np.array(bin_edges), bitrate, error_factor=error
    )
    label = f'{stn}_{int(100*error)}_error'
    count_df = pd.DataFrame(index=range(2**bitrate))
    count_df[label] = 0
    count_df[label] += fractional_obs_counts
    count_df.fillna(0, inplace=True)
    n_obs = np.nansum(count_df[label])
    # normalize p_obs and p_sim
    return count_df[label].values / n_obs
    

In [ ]:
def compute_unadjusted_counts(df, stn, bitrate):
    bin_edges = compute_log_uniform_bins(df, stn, bitrate)
    label = f'{stn}_simple_{bitrate}bits'
    df[label] = np.digitize(df[stn], bin_edges)
    # print(df[[stn, f'{stn}_quantized_{bitrate}bits']].head(4))
    # count the occurrences of each quantized value
    # the "simulated" series is the proxy/donor series
    # and the "observed" series is the target location
    obs_count_df = df.groupby(label).count()
    count_df = pd.DataFrame(index=range(2**bitrate))
    count_df[label] = 0
    count_df[label] += obs_count_df[stn]
    count_df.fillna(0, inplace=True)
    adjusted_p = count_df / obs_count_df[stn].sum()
    return adjusted_p.values.flatten()

In [ ]:
def compute_distortion(inputs):
    df, stn, b, err = inputs
    simple_frequencies = compute_unadjusted_counts(df, stn, b)
    error_adjusted_frequencies = apply_error_to_observations(df, stn, bitrate=b, error=err)
    # compute KL divergence between the simple and adjusted frequencies
    # this represents the distortion due to the error model
    mask = (simple_frequencies > 0) & (error_adjusted_frequencies > 0)
    distortion = np.zeros_like(simple_frequencies)
    distortion[mask] = simple_frequencies[mask] * np.log2(simple_frequencies[mask] / error_adjusted_frequencies[mask])
    kld = sum(distortion)
    return stn, kld, b, err

## Pairwise Processing



In [ ]:
import itertools

# generate all combinations of pairs of station ids
id_pairs = list(itertools.combinations(filtered_stns, 2))
print(f' There are {len(id_pairs)} unique pairings in the dataset')
# shuffle the pairs to make testing smaller batches more robust
np.random.seed(42)
np.random.shuffle(id_pairs)

In [ ]:
# load the attributes file with catchment geometries
geom_file = 'BCUB_watershed_attributes_updated.geojson'
bcub_gdf = gpd.read_file(os.path.join(os.getcwd(), 'data', geom_file))
bcub_gdf.columns = [c.lower() for c in bcub_gdf.columns]

In [ ]:
# set a revision date for the results output file
revision_date = '20241112'

# how many pairs to compute in each batch
batch_size = 5000
# batch_size = 10

# # what percentage of 365 observations in a year counts as a "complete" year
# completeness_threshold = 0.9
# min_observations = 365 * 0.9

# station pairs with less than min_years concurrent years of data are excluded (for concurrent analysis),
# stations with less than min_years are excluded (for non-concurrent analysis),
min_years = 1 #[2, 3, 4, 5, 10]

# a prior is applied to q in the form of a uniform array of 10**c pseudo-counts "c"
# this prior is used to test the effect of the choice of prior on the model
pseudo_counts = [-5, -4, -3, -2, -1, -0.5, -0.2, -0.1, 0, 0.1, 0.2, 0.5, 1, 2, 3, 4, 5]

# set the number of quantization levels to test, equal to 2^bitrate
bitrates = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

# Preload all records into a dictionary for fast lookup
records_dict = bcub_gdf.copy().set_index('official_id').to_dict(orient='index')

In [ ]:
def input_batch_generator(df, id_pairs_filtered, bitrate, 
                          min_years, use_partial_counts):
    batch_inputs = []
    for proxy, target in id_pairs_filtered:
        
        proxy_dict = records_dict.get(proxy, {})
        target_dict = records_dict.get(target, {})

        proxy_dict['official_id'] = proxy
        target_dict['official_id'] = target

        assert 'geometry' in proxy_dict.keys(), proxy_dict.keys()
        assert 'geometry' in target_dict.keys(), target_dict.keys()
        
        batch = [
            proxy_dict, target_dict, bitrate, 
            min_years,
        ]
        batch_inputs.append(batch)
    return batch_inputs

In [ ]:
temp_dir = os.path.join(os.getcwd(), 'data/', 'temp')
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

In [ ]:
def simulated_parametric_probabilities(bin_edges, proxy, target):
    """simulate the target using the parametric MLE parameters from the proxy"""
    ln_params = ln_dict[proxy.id]
    exp_params = expon_dict[proxy.id]
    p_sim = pd.DataFrame()
    bin_edges = [0] + bin_edges + [np.inf]

    # ln_pdf_vals = lognorm.pdf(bin_midpoints, ln_params['ln_shape'], loc=ln_params['ln_loc'], scale=ln_params['ln_scale'])
    # expon_pdf_vals = expon.pdf(bin_midpoints, loc=ln_params['ln_loc'], scale=ln_params['ln_scale'])
    ln_cdf_vals = lognorm.cdf(bin_edges, ln_params['ln_shape'], 
                              loc=ln_params['ln_loc'], scale=ln_params['ln_scale'])
    expon_cdf_vals = expon.cdf(bin_edges, loc=ln_params['ln_loc'], 
                               scale=ln_params['ln_scale'])
    # p_sim[target.ln_pdf_label] = ln_pdf_vals
    p_sim[target.ln_cdf_label] = np.diff(ln_cdf_vals)
    # p_sim[target.expon_pdf_label] = expon_pdf_vals
    p_sim[target.expon_cdf_label] = np.diff(expon_cdf_vals)

    # normalize the distributions
    p_sim /= p_sim.sum()
    assert np.isclose(p_sim.sum(), 1, atol=0.0001).all(), p_sim.sum()

    return p_sim

In [ ]:
def process_KL_divergence(p_obs, p_sim, bitrate, concurrent_data, epsilon=1e-12):
    """
    Processes the Kullback-Leibler (KL) divergence between observed and simulated probability distributions.

    Parameters
    ----------
    p_obs : np.ndarray
        The observed probability distribution.
    p_sim : pd.DataFrame
        A DataFrame containing the simulated probability distributions with different priors.
    bitrate : int
        The number of bits used for quantizing the observed series.
    concurrent_data : bool
        A flag indicating whether the data is concurrent.

    Returns
    -------
    pd.Series
        A series containing the sum of KL divergences for each simulated distribution.

    Raises
    ------
    Exception
        If any value in the simulated distribution is zero, which should not happen due to the addition of pseudo-counts.

    Notes
    -----
    - The function computes the KL divergence for each simulated distribution in `p_sim`.
    - It ensures that the probability distributions sum to 1 before computing the divergence.
    - If the data is concurrent, the divergence labels are prefixed with 'dkl_concurrent_', otherwise 'dkl_nonconcurrent_'.

    Example
    -------
    >>> p_obs = np.array([0.2, 0.3, 0.5])
    >>> p_sim = pd.DataFrame({'q_post_0.1R': [0.1, 0.4, 0.5], 'q_post_0.5R': [0.2, 0.3, 0.5]})
    >>> bitrate = 3
    >>> concurrent_data = True
    >>> sum_dkl = process_KL_divergence(p_obs, p_sim, bitrate, concurrent_data)
    >>> print(sum_dkl)
    """
    # dkl_df = uf.compute_kl_divergence(p_obs, p_sim, bitrate, concurrent_data)

    # explicitly set data types before vectorization
    p = np.array(p_obs, dtype=np.float64)
    mask = p > 0
    df = pd.DataFrame()
    df["bin"] = range(1, 2**bitrate + 1)
    df.set_index("bin", inplace=True)
    small_val_flags = []
    for c in p_sim.columns:
        label = "dkl_nonconcurrent_" + "_".join(c.split("_")[1:])
        if concurrent_data is True:
            label = "dkl_concurrent_" + "_".join(c.split("_")[1:])
        q = np.array(p_sim[c].values, dtype=np.float64) 
        small_values = q < epsilon
        kld_array = np.zeros_like(p)
        q = np.clip(q, epsilon, None)
        kld_array[mask] = p[mask] * np.log2(p[mask] / q[mask])
        df[label] = kld_array
        if np.any(small_values):
            small_val_flags.append(c)
    n_flags = len(small_val_flags)
    q_flag = None
    if n_flags > 0:
        q_flag = ','.join(small_val_flags)
        # print(q_flag)
    sum_dkl = df.sum()

    if np.any(sum_dkl.values <= 0):
        print(f"negative or zero dkl")
        print(sum_dkl.values)
        raise Exception('negative or zero dkl')

    return sum_dkl, q_flag

In [ ]:
def process_divergences(result, p_obs, p_sim, bin_edges, bitrate, concurrent_data):
    dkl, q_flag = process_KL_divergence(p_obs, p_sim, bitrate, concurrent_data)

    # p = p_obs
    # q = p_sim["q_sim_no_prior"].values
    # q_uniform = p_sim["q_uniform"].values
    result.update(dkl.to_dict())
    result['small_q_flag'] = q_flag

    return result

In [ ]:
def process_probabilities(
    df, proxy, target, bitrate, concurrent_data, #pseudo_counts, p_errors
):
    target.obs_quantized_label = f"obs_quantized_{target.id}_{bitrate}b"
    target.sim_quantized_label = f"sim_quantized_{target.id}_{bitrate}b"
    # compute the bin edges based on equal width in log space
    # binning should be done on the 'ground truth' observed range (the target)
    bin_edges = dpf.uniform_log_bins(df, target, bitrate)

    simple_count_df = dpf.compute_unadjusted_counts(
        df, target, bin_edges, bitrate, concurrent_data
    )
    
    p_obs = simple_count_df[target.obs_label].values / simple_count_df[target.obs_label].sum()

    # add a uniformly distributed error to the observed data
    # and compute probabilities from partial observation counts
    # where counts are divided based on the proportion of the bin
    # that the measurement error falls within
    fractional_obs_counts = dpf.error_adjusted_fractional_bin_counts(
        df[target.obs_label], np.array(bin_edges), bitrate, error_factor=0.1
    )
    fractional_sim_counts = dpf.error_adjusted_fractional_bin_counts(
        df[target.sim_label], np.array(bin_edges), bitrate, error_factor=0.1
    )

    t1 = time()
    # print(f' {t1-t0:.2f}s to process fractional bin counts')

    partial_count_df = pd.DataFrame(index=range(2**bitrate))
    partial_count_df[target.obs_label] = 0
    partial_count_df[target.sim_label] = 0
    partial_count_df[target.obs_label] += fractional_obs_counts
    partial_count_df[target.sim_label] += fractional_sim_counts
    partial_count_df.fillna(0, inplace=True)

    partial_count_df /= partial_count_df.sum()
    
    # Check if the sums are close enough to 1 within a tolerance of 0.001
    assert np.isclose(partial_count_df.sum(), 1, atol=0.0001).all(), partial_count_df.sum()

    p_sim = simulated_parametric_probabilities(bin_edges, proxy, target)
    
    return p_obs, p_sim, bin_edges

In [ ]:
def process_batch(inputs):    
    (
        proxy,
        target,
        bitrate,
        min_concurrent_years,
    ) = inputs
    
    proxy_id, target_id = proxy['official_id'], target['official_id']
    bitrate = int(bitrate)

    # create a result dict object for tracking results of the batch comparison
    result = {
        "proxy": proxy_id,
        "target": target_id,
        "bitrate": bitrate,
        "min_concurrent_years": min_concurrent_years,
    }
    station_info = {"proxy": proxy, "target": target}

    # check if the polygons are nested
    result["nested_catchments"] = dpf.check_if_nested(
        proxy, target
    )

    # for stn in pair:
    proxy = dpf.Station(station_info["proxy"])
    target = dpf.Station(station_info["target"])
    target.ln_pdf_label = f'{target.id}_sim_lognorm_pdf'
    target.ln_cdf_label = f'{target.id}_sim_lognorm_cdf'
    target.expon_pdf_label = f'{target.id}_sim_expon_pdf'
    target.expon_cdf_label = f'{target.id}_sim_expon_cdf'

    # compute spatial distance
    p1, p2 = (
        station_info["proxy"]["geometry"].centroid,
        station_info["target"]["geometry"].centroid,
    )
    # compute the distance between catchment centroids (km)
    centroid_distance = p1.distance(p2) / 1000
    result["centroid_distance"] = round(centroid_distance, 2)
    if centroid_distance > 1000:
        return None

    if np.isnan(target.drainage_area_km2):
        raise ValueError(f"No drainage area for {target_id}")
    if np.isnan(proxy.drainage_area_km2):
        raise ValueError(f"No drainage area for {proxy_id}")

    # Retrieve the data for both stations
    # this is all data, including non-concurrent
    adf = dpf.retrieve_nonconcurrent_data(proxy_id, target_id)

    assert ~adf.empty, "No data returned."

    for stn in [proxy, target]:
        adf = dpf.transform_and_jitter(adf, stn)

    # simulate flow at the target based on equal unit area runoff scaling
    adf[target.sim_label] = adf[proxy.id] * (
        target.drainage_area_km2 / proxy.drainage_area_km2
    )

    # filter for the concurrent data
    df = adf.copy().dropna(subset=[proxy_id, target_id], how="any")
    result["num_concurrent_obs"] = len(df)
    
    if df.empty:
        num_complete_concurrent_years = 0
    else:
        df.reset_index(inplace=True)
        num_complete_concurrent_years = dpf.count_complete_years(df, 'time', proxy_id)
        
    counts = df[[proxy_id, target_id]].count(axis=0)
    counts = adf.count(axis=0)
    proxy.n_obs, target.n_obs = counts[proxy_id], counts[target_id]
    result[f"proxy_n_obs"] = proxy.n_obs
    result[f"target_n_obs"] = target.n_obs
    result[f"proxy_frac_concurrent"] = len(df) / proxy.n_obs
    result[f"target_frac_concurrent"] = len(df) / target.n_obs

    if (counts[proxy_id] == 0) or (counts[target_id] == 0):
        print(f"   Zero observations.  Skipping.")
        return None

    # process the PMFs and divergences for concurrent data
    # using a range of uniform priors via pseudo counts
    if num_complete_concurrent_years > min_concurrent_years:
        # df is concurrent data, so the results
        # are updating concurrent data here
        # df, proxy, target, bitrate, concurrent_data, partial_counts, pseudo_counts
        concurrent_data = True
        p_obs, p_sim, bin_edges = process_probabilities(
            df, proxy, target, bitrate, concurrent_data, #pseudo_counts, p_errors
        )
    if (target.n_obs > 365 * 0.9) & (proxy.n_obs > 365 * 0.9):
        # adf is all data (includes non-concurrent), so the results
        # are updated if both series meet the minimum length
        concurrent_data = False
        p_obs, p_sim, bin_edges = process_probabilities(
            adf, proxy, target, bitrate, concurrent_data,# pseudo_counts, p_errors
        )
        
    result = process_divergences(
        result, p_obs, p_sim, bin_edges, bitrate, concurrent_data
    )

    noise = process_noise(result, p_obs, p_sim, bin_edges, bitrate, concurrent_data)
    # result['underspecified_model_flag'] = underspecified_flag
    return result

In [ ]:
# the 'process' variable is here so jupyter doesn't go computing 
# a million rows per iteration when the book is built for pushing to github pages.
# reordered_bitrates = [4, 6, 8, 10, 12, 3, 5, 7, 9, 11]
process = True
partial_counts = False
if process: 
    for bitrate in reordered_bitrates:
        print(f'Processing pairs at {bitrate} bits quantization ')
        results_fname = f'KL_parametric_fits_{bitrate}bits_{revision_date}.csv'

        out_fpath = os.path.join('data/', 'parametric_divergence_test', results_fname)
        if os.path.exists(out_fpath):
            continue

        n_batches = max(len(id_pairs) // batch_size, 1)
        batches = np.array_split(np.array(id_pairs, dtype=object), n_batches)
        n_pairs = len(id_pairs)
        print(
            f"    Processing {n_pairs} pairs in {n_batches} batches at {bitrate} bits"
        )
        batch_no = 1
        batch_files = []
        t0 = time()
        # error_df = error_model_df[error_model_df['bitrate'] == bitrate].copy()
        for batch_ids in batches:
            print(f'Starting batch {batch_no}/{len(batches)} processing.')
            batch_fname = results_fname.replace('.csv', f'_batch_{batch_no:03d}.csv')
            batch_output_fpath = os.path.join(temp_dir, batch_fname)
            if os.path.exists(batch_output_fpath):
                batch_files.append(batch_output_fpath)
                batch_no += 1
                continue
            
            # define the input array for multiprocessing
            inputs = input_batch_generator(bcub_gdf, batch_ids, bitrate,
                     min_years, partial_counts)

            with mp.Pool(20) as pool:
                results = pool.map(process_batch, inputs)
                results = [r for r in results if r is not None]

            batch_result = pd.DataFrame(results)
            if batch_result.empty:
                print('Empty batch.  Skipping')
            else:
                batch_result.to_csv(batch_output_fpath, index=False)
                print(f"    Saved {len(batch_result)} new results to file.")
            
            batch_files.append(batch_output_fpath)
            t2 = time()
            print(f'    Processed {len(batch_ids)} pairs at ({bitrate} bits) in {t2 - t0:.1f} seconds')
            batch_no += 1
            
        print(f'    Concatenating {len(batch_files)} batch files.')
        if len(batch_files) > 0:
            all_results = pd.concat([pd.read_csv(f, engine='pyarrow') for f in batch_files], axis=0)
            all_results.to_csv(out_fpath, index=False)
            if os.path.exists(out_fpath):
                for f in batch_files:
                    os.remove(f)
            print(f'    Wrote {len(all_results)} results to {out_fpath}')
        else:
            print('    No new results to write to file.')

## Citations

```{bibliography}
:filter: docname in docnames
```